In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import re
import pickle
import modules.utils as Utils
from modules.regex_patterns import reviews_paths
from collections import Counter
from underthesea import word_tokenize
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

### 20. Load file lên và shuffle nó

In [3]:
data = pd.read_csv("./data/standard_data.csv")

In [4]:
data = shuffle(data)

In [5]:
data

,raw_comment,standard_comment,label
60357,"Chất vài khá tốt, mặc khá là thoải mái. Sẽ ủng...",chất vài khá thoải_mái tiếp ủng_hộ khá_tốt,1
71678,nhan dep hoi rong goi hang chac chan ky cang n...,nhan hoi rong goi hang chan ky cang noi la ung,1
153505,"Kẹp cực xinh, giống hình, màu đậm hơn hình một...",kẹp cực xinh đậm hình xíu giống_hình,1
165989,"Thời gian giao hơn 10 ngày. Đầy đủ hàng. Đẹp, ...",thời_gian ngày đầy_đủ đẹp hình nên_mua như_hìn...,1
87145,Quá nhiều chỉ thừa và đường may kém nhưng lên ...,nhiều thừa kém đẹp hơi vọng góp_ý cải_thiện ch...,0
...,...,...,...
51959,Áo đẹp lắm mn. Vải ko mỏng lắm đâu. Bth....,a o đe mọi va i mo bo co ne giư cô a o mọi_ngư...,1
184097,Ví đẹp nhưng hơi có mùi da,ví đẹp hơi mùi da,1
66867,Đóng gói cẩn thận giao hàng nhanh. Hàng chất l...,nhanh vớ vẻ nhỏ so chân nam chưa thử giao_hàng...,1
11382,Săn sale mà rất hài lòng luôn ấy ạ. Shop giao ...,săn hài_lòng ấy nhanh dịch_vụ ổn giao_nhanh sale,1


### 21. Kiểm tra số mẫu giữa các nhóm

In [6]:
no_samples_each_group = data[['label', 'standard_comment']].groupby(['label']).count()

In [7]:
no_samples_each_group

,standard_comment
label,
-1,3458
0,11845
1,204293


> **Nhận xét**:
> * Số lượng mẫu giữa các group không đều.
>
> $\Rightarrow$ thực hiện oversampling

### 22. Tạo tập `final_data`

In [8]:
no_samples_each_group = no_samples_each_group['standard_comment'].to_list()

In [9]:
negative = data.label[data.label.eq(-1)].sample(no_samples_each_group[0]).index
normal = data.label[data.label.eq(0)].sample(no_samples_each_group[1]).index
positive = data.label[data.label.eq(1)].sample(no_samples_each_group[1] + no_samples_each_group[0]*2).index

final_data = data.loc[negative.union(normal).union(positive)]

In [10]:
final_data = shuffle(final_data)

In [11]:
final_data

,raw_comment,standard_comment,label
1156,Chất vải phù hợp giá tiền rất ok,phù_hợp giá_tiền chất_vải,1
211787,Hàng về hơi lâu 1 chút. Nhưng đẹp miễn bàn đán...,chút đẹp miễn bàn đáng công chờ_đợi đáng iu cự...,1
79513,Chất qua ok so vs 170k nhé ăee,chất qua so,1
88604,"Áo khá đẹp nha, form rộng đẹp. Mỗi tội chất vả...",rộng đẹp mỗi tội bí lười chụp ảnh khum liên_qu...,1
143505,Chất lượng xứng với giá tiền nhưng lần này gia...,xứng lần chậm ơi chậm giao_hàng_chậm giá_tiền ...,1
...,...,...,...
280,hâu,hâu,0
89981,"Dùng tạm ảnh này ạ\nChất liệu được, đường may ...",dùng tạm ảnh chất_liệu chắc_chắn sổ không_chắc...,0
119346,"Áo đẹp,nên mua ạ",đẹp nên_mua áo_đẹp,1
106099,"Hàng sale chất ổn nha, 2 đôi 2 màu, giao hàng ...",chất ổn đôi nhanh giao_hàng_nhanh cẩn_thận đón...,1


In [12]:
final_data = final_data.reset_index(drop=True)

### 23. Thực hiện TF-IDF với `ngram_range=(1, 4)`

In [13]:
tf_idf = TfidfVectorizer(ngram_range=(1, 4))
tf_idf.fit(final_data['standard_comment'])

TfidfVectorizer(ngram_range=(1, 4))

### 24. Tạo tập input `X` và output `y`

In [14]:
X = tf_idf.transform(final_data['standard_comment'])
y = final_data['label']

### 25. Thực hiện over-sampling

In [15]:
ros = RandomOverSampler()
X_ros, y_ros = ros.fit_resample(X, y)

In [16]:
Counter(y_ros)

Counter({1: 18761, -1: 18761, 0: 18761})

### 26. Train test split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_ros, y_ros, test_size=0.2, random_state=100, shuffle=True)

### 27. Lưu lại `final_data`, `tf_idf`, `X_train`, `X_test`, `y_train`, `y_test`

In [18]:
final_data.to_csv("./data/final_data.csv", index=False)

In [19]:
pickle.dump(tf_idf, open("./data/tf_idf.pickle", "wb"))
pickle.dump(X_train, open("./data/X_train.pickle", "wb"))
pickle.dump(X_test, open("./data/X_test.pickle", "wb"))
pickle.dump(y_train, open("./data/y_train.pickle", "wb"))
pickle.dump(y_test, open("./data/y_test.pickle", "wb"))